# 310_Join_FIFA_Fantasy1617 
## Purpose 
In this notebook we read both our Fantasty and FIFA cleaned datasets and join them and performing any necessary cleaning.  
## Datasets 
* _Input_: FIFA18.csv,Fantasy1617.csv
* _Output_: Joined1617.csv

In [1]:
import os.path
import numpy as np
import pandas as pd

### Reading in our cleaned FIFA and Fantasy datasets.

In [2]:
Fifa = pd.read_csv("../../data/prep/FIFA18.csv")
Fantasy1617 = pd.read_csv("../../data/prep/Fantasy1617.csv")

### Merging the two datasets on Surname and First Initial
* Use pd.merge to join the tables  .
* Drop irrelevant column 'Unnamed: 0_x'.

In [3]:
Join1617 = pd.merge(Fifa, Fantasy1617, on=['Surname','FirstInitial'])
Join1617.drop(Join1617.columns[[0]], axis=1, inplace=True)

In [4]:
Join1617.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2228,32,Spanish Primera Division,1985-02-05,185.0,...,0,5,-,1,-,-,0,29,6.724138,Cristiano
1,158023,L. Messi,Lionel Messi,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2158,30,Spanish Primera Division,1987-06-24,170.0,...,2,4,-,1,-,-,0,34,7.941176,Lionel
2,190871,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2100,25,French Ligue 1,1992-02-05,175.0,...,0,2,-,0,-,-,0,30,4.800000,Neymar
3,176580,L. Suarez,Luis Suarez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera Division,1987-01-24,182.0,...,2,9,-,0,-,-,0,36,6.694444,Luis
4,176580,L. Suarez,Luis Suarez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera Division,1987-01-24,182.0,...,3,0,-,0,-,-,0,3,1.000000,Leonardo


### Creating a while loop to remove players
* Removes all players thats Firstname is not contained in 'name'
* Removes all players thats Firstname is not contained in 'full_name'.

In [5]:
i = 0
array = []

while i < len(Join1617):
    if Join1617['Firstname'][i] in Join1617['name'][i] or Join1617['Firstname'][i] in Join1617['full_name'][i]:
        array.append(True)
        i+=1
    else:
        array.append(False)
        i+=1

### Making the array 'array' a numpy array 
* Making the numpy array into a dataframe.

In [6]:
array = np.array(array)
Join1617 = Join1617[array]

### Creating a dataframe ERROR with all players that have problems
* Contains players in Join that are duplicated using 'Players' or 'full_name'.

In [7]:
Error1617 = Join1617[(Join1617['Players'].isin(Join1617['Players'][Join1617['Players'].duplicated()]) | Join1617['full_name'].isin(Join1617['full_name'][Join1617['full_name'].duplicated()]))]
Error1617.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
48,176676,Marcelo,Marcelo Vieira da Silva,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2214,29,Spanish Primera Division,1988-05-12,174.0,...,4,3,-,0,34,11,0,30,3.366667,Marcelo
49,180334,Marcelo,Marcelo Antonio Guedes Filho,Olympique Lyonnais,https://cdn.sofifa.org/18/teams/66.png,1789,30,French Ligue 1,1987-05-20,191.0,...,4,3,-,0,34,11,0,30,3.366667,Marcelo
50,205340,Marcelo,Marcelo dos Santos Ferreira,Rio Ave FC,https://cdn.sofifa.org/18/teams/744.png,1625,27,Portuguese Primeira Liga,1989-07-27,182.0,...,4,3,-,0,34,11,0,30,3.366667,Marcelo
53,168542,David Silva,David Josue Jimenez Silva,Manchester City,https://cdn.sofifa.org/18/teams/10.png,1977,31,English Premier League,1986-01-08,173.0,...,3,11,-,0,-,-,0,34,3.029412,David
57,214378,D. Silva,David Silva,CD Los Millionarios Bogota,https://cdn.sofifa.org/18/teams/101105.png,1939,30,Colombian Primera A,1986-12-13,175.0,...,3,11,-,0,-,-,0,34,3.029412,David


### Creating a dataframe TopLeagues with all players inside the Top 5 leagues
* Takes all players in ERROR insode the Top 5 leagues using str.contains.

In [8]:
leagues = ['German Bundesliga','Spanish Primera Division','English Premier League','French Ligue 1','Italian Serie A']
TopLeagues1617 = Error1617[(Error1617.league.str.contains('|'.join(leagues)) == True) | Error1617['league'].isnull()]

### Creating a dataframe DELETE with all players outside the Top 5 leagues
* Takes all players in ERROR outside the Top 5 leagues using str.contains.

In [9]:
Delete1617 = Error1617[Error1617.league.str.contains('|'.join(leagues)) == False]

### Creating a dataframe FIX
* This dataframe takes all players in the Top 5 that have been duplicated .

In [10]:
Fix1617 = TopLeagues1617[(TopLeagues1617['Players'].isin(TopLeagues1617['Players'][TopLeagues1617['Players'].duplicated()]) | TopLeagues1617['full_name'].isin(TopLeagues1617['full_name'][TopLeagues1617['full_name'].duplicated()]))]
Fix1617.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
48,176676,Marcelo,Marcelo Vieira da Silva,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2214,29,Spanish Primera Division,1988-05-12,174.0,...,4,3,-,0,34,11,0,30,3.366667,Marcelo
49,180334,Marcelo,Marcelo Antonio Guedes Filho,Olympique Lyonnais,https://cdn.sofifa.org/18/teams/66.png,1789,30,French Ligue 1,1987-05-20,191.0,...,4,3,-,0,34,11,0,30,3.366667,Marcelo
118,179944,David Luiz,David Luiz Moreira Marinho,Chelsea,https://cdn.sofifa.org/18/teams/5.png,2123,30,English Premier League,1987-04-22,189.0,...,0,2,-,0,26,8,0,33,3.424242,David
119,179944,David Luiz,David Luiz Moreira Marinho,Chelsea,https://cdn.sofifa.org/18/teams/5.png,2123,30,English Premier League,1987-04-22,189.0,...,0,1,-,0,3,2,0,3,3.666667,David
187,202166,J. Draxler,Julian Draxler,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2017,23,French Ligue 1,1993-09-20,187.0,...,1,5,-,0,-,-,0,13,2.384615,Julian


### Removing players from JOIN
* Removing players whos full name is in Delete and have the same birth date.
* Removing players whos full name is in Fix and have the same birth date.

In [11]:
Join1617 = Join1617[(~(Join1617.full_name.isin(Delete1617.full_name) & Join1617.birth_date.isin(Delete1617.birth_date)))]
Join1617 = Join1617[(~(Join1617.full_name.isin(Fix1617.full_name) & Join1617.birth_date.isin(Fix1617.birth_date)))]

### Removing players from FIX
* Removing players thats 'full_name' is not equal to 'Players'.
* Dropping all duplicate players in Fixed.

In [12]:
Fixed1617 = Fix1617[Fix1617['full_name'] == Fix1617['Players']]
Fixed1617 = Fixed1617[(Fixed1617['Players'].isin(Fixed1617['Players'][Fixed1617['Players'].duplicated()]) | Fixed1617['full_name'].isin(Fixed1617['full_name'][Fixed1617['full_name'].duplicated()]))]
Fixed1617 = Fixed1617[~Fixed1617.duplicated(['Players', 'Clubs', 'Points'])]
Fixed1617.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
187,202166,J. Draxler,Julian Draxler,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2017,23,French Ligue 1,1993-09-20,187.0,...,1,5,-,0,-,-,0,13,2.384615,Julian
188,202166,J. Draxler,Julian Draxler,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2017,23,French Ligue 1,1993-09-20,187.0,...,5,9,-,0,-,-,0,17,3.000000,Julian
193,192227,S. Mustafi,Shkodran Mustafi,Arsenal,https://cdn.sofifa.org/18/teams/1.png,1942,25,English Premier League,1992-04-17,184.0,...,0,1,-,0,31,13,0,26,2.923077,Shkodran
194,192227,S. Mustafi,Shkodran Mustafi,Arsenal,https://cdn.sofifa.org/18/teams/1.png,1942,25,English Premier League,1992-04-17,184.0,...,1,0,-,0,4,3,0,1,-2.000000,Shkodran
270,177388,D. Payet,Dimitri Payet,Olympique de Marseille,https://cdn.sofifa.org/18/teams/219.png,2062,30,French Ligue 1,1987-03-29,175.0,...,1,5,-,0,-,-,0,18,3.388889,Dimitri


### Adding players who have transfered teams
* Get the sum of all relevant columns to get one full player

In [13]:
d = {
 'ID' : 'first',
 'club_logo':'first',
 'special':'first',
 'age':'first',
 'league':'first',
 'birth_date':'first',
 'height_cm':'first',
 'weight_kg':'first',
 'body_type':'first',
 'real_face':'first',
 'flag':'first',
 'nationality':'first',
 'photo':'first',
 'eur_value':'first',
 'eur_wage':'first',
 'eur_release_clause':'first',
 'overall':'first',
 'potential':'first',
 'pac':'first',
 'sho':'first',
 'pas':'first',
 'dri':'first',
 'def':'first',
 'phy':'first',
 'international_reputation':'first',
 'skill_moves':'first',
 'weak_foot':'first',
 'work_rate_att':'first',
 'work_rate_def':'first',
 'preferred_foot':'first',
 'crossing':'first',
 'finishing':'first',
 'heading_accuracy':'first',
 'short_passing':'first',
 'volleys':'first',
 'dribbling':'first',
 'curve':'first',
 'free_kick_accuracy':'first',
 'long_passing':'first',
 'ball_control':'first',
 'acceleration':'first',
 'sprint_speed':'first',
 'agility':'first',
 'reactions':'first',
 'balance':'first',
 'shot_power':'first',
 'jumping':'first',
 'stamina':'first',
 'strength':'first',
 'long_shots':'first',
 'aggression':'first',
 'interceptions':'first',
 'positioning':'first',
 'vision':'first',
 'penalties':'first',
 'composure':'first',
 'marking':'first',
 'standing_tackle':'first',
 'sliding_tackle':'first',
 'gk_diving':'first',
 'gk_handling':'first',
 'gk_kicking':'first',
 'gk_positioning':'first',
 'gk_reflexes':'first',
 'rs':'first',
 'rw':'first',
 'rf':'first',
 'ram':'first',
 'rcm':'first',
 'rm':'first',
 'rdm':'first',
 'rcb':'first',
 'rb':'first',
 'rwb':'first',
 'st':'first',
 'lw':'first',
 'cf':'first',
 'cam':'first',
 'cm':'first',
 'lm':'first',
 'cdm':'first',
 'cb':'first',
 'lb':'first',
 'lwb':'first',
 'ls':'first',
 'lf':'first',
 'lam':'first',
 'lcm':'first',
 'ldm':'first',
 'lcb':'first',
 'gk':'first',
 '1_on_1_rush_trait':'first',
 'acrobatic_clearance_trait':'first',
 'argues_with_officials_trait':'first',
 'avoids_using_weaker_foot_trait':'first',
 'backs_into_player_trait':'first',
 'bicycle_kicks_trait':'first',
 'cautious_with_crosses_trait':'first',
 'chip_shot_trait':'first',
 'chipped_penalty_trait':'first',
 'comes_for_crosses_trait':'first',
 'corner_specialist_trait':'first',
 'diver_trait':'first',
 'dives_into_tackles_trait':'first',
 'diving_header_trait':'first',
 'driven_pass_trait':'first',
 'early_crosser_trait':'first',
 "fan's_favourite_trait":'first',
 'fancy_flicks_trait':'first',
 'finesse_shot_trait':'first',
 'flair_trait':'first',
 'flair_passes_trait':'first',
 'gk_flat_kick_trait':'first',
 'gk_long_throw_trait':'first',
 'gk_up_for_corners_trait':'first',
 'giant_throw_in_trait':'first',
 'inflexible_trait':'first',
 'injury_free_trait':'first',
 'injury_prone_trait':'first',
 'leadership_trait':'first',
 'long_passer_trait':'first',
 'long_shot_taker_trait':'first',
 'long_throw_in_trait':'first',
 'one_club_player_trait':'first',
 'outside_foot_shot_trait':'first',
 'playmaker_trait':'first',
 'power_free_kick_trait':'first',
 'power_header_trait':'first',
 'puncher_trait':'first',
 'rushes_out_of_goal_trait':'first',
 'saves_with_feet_trait':'first',
 'second_wind_trait':'first',
 'selfish_trait':'first',
 'skilled_dribbling_trait':'first',
 'stutter_penalty_trait':'first',
 'swerve_pass_trait':'first',
 'takes_finesse_free_kicks_trait':'first',
 'target_forward_trait':'first',
 'team_player_trait':'first',
 'technical_dribbler_trait':'first',
 'tries_to_beat_defensive_line_trait':'first',
 'poacher_speciality':'first',
 'speedster_speciality':'first',
 'aerial_threat_speciality':'first',
 'dribbler_speciality':'first',
 'playmaker_speciality':'first',
 'engine_speciality':'first',
 'distance_shooter_speciality':'first',
 'crosser_speciality':'first',
 'free_kick_specialist_speciality':'first',
 'tackling_speciality':'first',
 'tactician_speciality':'first',
 'acrobat_speciality':'first',
 'strength_speciality':'first',
 'clinical_finisher_speciality':'first',
 'prefers_rs':'first',
 'prefers_rw':'first',
 'prefers_rf':'first',
 'prefers_ram':'first',
 'prefers_rcm':'first',
 'prefers_rm':'first',
 'prefers_rdm':'first',
 'prefers_rcb':'first',
 'prefers_rb':'first',
 'prefers_rwb':'first',
 'prefers_st':'first',
 'prefers_lw':'first',
 'prefers_cf':'first',
 'prefers_cam':'first',
 'prefers_cm':'first',
 'prefers_lm':'first',
 'prefers_cdm':'first',
 'prefers_cb':'first',
 'prefers_lb':'first',
 'prefers_lwb':'first',
 'prefers_ls':'first',
 'prefers_lf':'first',
 'prefers_lam':'first',
 'prefers_lcm':'first',
 'prefers_ldm':'first',
 'prefers_lcb':'first',
 'prefers_gk':'first',
 'Players':'first',
 'Position':'first',
 'Clubs':'first',
 'Points':'sum',  
 'Start':'sum', 
 'Sub':'sum', 
 'club':'last',
 'Position':'first',
 'Start':'sum',
 'Goals':'sum',
 'A':'sum',
 'CS':'sum',
 'CS part':'sum',
 'Yellow':'sum',
 'Red':'sum',
 'Sub':'sum',
 'CO':'sum',
 'Off':'sum',
 'Pen SV':'sum',
 'Pen M':'sum',
 'Goals conceded':'sum',
 'Conceded 1+':'sum',
 'OG':'sum',
 'Firstname':'first',
 'Apps':'sum',
 'Form': 'mean'
}

#### Groupby function applying 'd' to fix joining the players.

In [14]:
Fixed1617 = (Fixed1617.groupby(['name','full_name'], sort=False, as_index=False).agg(d).reindex(columns=Fix1617.columns))
Fixed1617.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
0,202166,J. Draxler,Julian Draxler,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2017,23,French Ligue 1,1993-09-20,187.0,...,6,14,--,0,--,--,0,30,2.692308,Julian
1,192227,S. Mustafi,Shkodran Mustafi,Arsenal,https://cdn.sofifa.org/18/teams/1.png,1942,25,English Premier League,1992-04-17,184.0,...,1,1,--,0,314,133,0,27,0.461538,Shkodran
2,177388,D. Payet,Dimitri Payet,Olympique de Marseille,https://cdn.sofifa.org/18/teams/219.png,2062,30,French Ligue 1,1987-03-29,175.0,...,2,9,--,0,--,--,0,33,3.527778,Dimitri
3,174543,C. Bravo,Claudio Bravo,Manchester City,https://cdn.sofifa.org/18/teams/10.png,1464,34,English Premier League,1983-04-13,184.0,...,0,1,10,0,262,101,0,23,1.886364,Claudio
4,167431,G. Castro,Gonzalo Castro,Borussia Dortmund,https://cdn.sofifa.org/18/teams/22.png,2156,30,German Bundesliga,1987-06-11,172.0,...,5,10,-,0,-,-,0,28,2.821429,Gonzalo


#### Quick look at what FIXED with most relevant columns

In [15]:
Fixed1617[['full_name','club','Players','Clubs','Points']].head(10)

,full_name,club,Players,Clubs,Points
0,Julian Draxler,Paris Saint-Germain,Julian Draxler,VFL,82
1,Shkodran Mustafi,Arsenal,Shkodran Mustafi,ARSL,74
2,Dimitri Payet,Olympique de Marseille,Dimitri Payet,WESTHAM,116
3,Claudio Bravo,Manchester City,Claudio Bravo,MCFC,62
4,Gonzalo Castro,Borussia Dortmund,Gonzalo Castro,BD1,79
5,Joe Hart,West Ham United,Joe Hart,MCFC,59
6,Simone Zaza,Valencia CF,Simone Zaza,WESTHAM,72
7,Stevan Jovetic,AS Monaco,Stevan Jovetic,SEVI,73
8,Clement Lenglet,Sevilla FC,Clement Lenglet,SEVI,98
9,Memphis Depay,Olympique Lyonnais,Memphis Depay,MUFC,81


### Joining Join and Fixed
* Using pd.concat to join the two dataframes together.
* Drop irrelevant column.

In [16]:
Joined1617 = pd.concat([Join1617,Fixed1617])
Joined1617 = Joined1617.reset_index()
Joined1617.drop(Joined1617.columns[[0]], axis=1, inplace=True)
Joined1617.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2228,32,Spanish Primera Division,1985-02-05,185.0,...,0,5,-,1,-,-,0,29,6.724138,Cristiano
1,158023,L. Messi,Lionel Messi,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2158,30,Spanish Primera Division,1987-06-24,170.0,...,2,4,-,1,-,-,0,34,7.941176,Lionel
2,190871,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2100,25,French Ligue 1,1992-02-05,175.0,...,0,2,-,0,-,-,0,30,4.800000,Neymar
3,176580,L. Suarez,Luis Suarez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera Division,1987-01-24,182.0,...,2,9,-,0,-,-,0,36,6.694444,Luis
4,225591,L. Suarez,Leonardo Suarez,Villarreal CF,https://cdn.sofifa.org/18/teams/483.png,1673,21,Spanish Primera Division,1996-03-30,167.0,...,3,0,-,0,-,-,0,3,1.000000,Leonardo


#### Saving to csv file in data/prep

In [17]:
Joined1617.to_csv('../../data/prep/Joined1617.csv')